In [1]:
import os
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import cv2
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
folder_path = '/kaggle/input/clahe-images/final_images/final_images'  # Replace with your folder path

In [3]:
folders=sorted(os.listdir(folder_path))

In [4]:
trainpaths=[]
for i in tqdm(range(len(folders))):
    files=os.listdir(os.path.join(folder_path,folders[i]))
    files.sort(key=lambda x: int(x.split('.')[0]))
    files=[os.path.join(folder_path,folders[i],files[j]) for j in range(len(files))]
    trainpaths=trainpaths+files

100%|██████████| 27/27 [00:03<00:00,  6.86it/s]


In [5]:
batch_size=32
num_batches = len(trainpaths) // batch_size
batches = np.empty((num_batches, batch_size), dtype=object)

for i in range(num_batches):
    start = i * batch_size
    end = start + batch_size
    batches[i] = trainpaths[start:end]

# If there are remaining paths, you can handle them separately
remaining_paths = trainpaths[num_batches * batch_size:]
batches.shape,len(remaining_paths)

((3506, 32), 6)

In [6]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import torch

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')

2024-08-03 07:27:14.920430: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 07:27:14.920536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 07:27:15.043327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [7]:
import tensorflow as tf
import time 
a=time.time()
feat=[]
for i in tqdm(range(num_batches)):
    image_paths = batches[i]
    images = [Image.open(image_path).convert('RGB') for image_path in image_paths]
    inputs = processor(images=images, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    last_hidden_states_np = last_hidden_states.detach().numpy()
    outputs1 = tf.reduce_mean(last_hidden_states_np, axis=1)
    feat.append(outputs1)
if True:
    image_paths = remaining_paths
    images = [Image.open(image_path).convert('RGB') for image_path in image_paths]
    inputs = processor(images=images, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    last_hidden_states_np = last_hidden_states.detach().numpy()
    outputs1 = tf.reduce_mean(last_hidden_states_np, axis=1)
    feat.append(outputs1)

100%|██████████| 3506/3506 [7:10:16<00:00,  7.36s/it]


In [8]:
a=feat
concatenated_tensor = tf.concat(a, axis=0)
print(concatenated_tensor.shape)  # Should output (64, 768)

(112198, 768)


In [9]:
tensor_array = concatenated_tensor.numpy()
# Save as a .npy file
np.save('tensor.npy', tensor_array)
# features=np.load('/kaggle/input/vit-feat/tensor.npy')
# labels=pd.read_csv('/kaggle/input/clahe-images/labels_path.csv')